# 💼 Build a TariffIQ Chat Agent — Haystack x OpenAI

📚 Welcome to this hands-on hackathon notebook! 
In this tutorial, you'll build a **chat agent that answers real-world tariff-related questions**, product exemptions, and executive order impacts — using Haystack and OpenAI's function calling.

### 💡 Why This Matters:
Global trade is complex. This app helps businesses **ask questions** and get **answers** from structured data and real documents.

---
### 1. 🧰 What You’ll Learn:
In this tutorial, you'll learn how to convert your Haystack pipeline into a function-calling tool and how to implement applications using OpenAI's Chat Completion API through `OpenAIChatGenerator` for agent-like behavior.

- Embed documents with Haystack
- Build a Retrieval-Augmented Generator (RAG)
- Use OpenAI to call functions (aka tools)
- Deploy a smart chatbot via Gradio


### Tutorial Info
- **Level**: Advanced
- **Time to complete**: 20 minutes
- **Components Used**: [InMemoryDocumentStore](https://docs.haystack.deepset.ai/docs/inmemorydocumentstore), [SentenceTransformersDocumentEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformersdocumentembedder), [SentenceTransformersTextEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformerstextembedder), [InMemoryEmbeddingRetriever](https://docs.haystack.deepset.ai/docs/inmemoryembeddingretriever), [ChatPromptBuilder](https://docs.haystack.deepset.ai/docs/chatpromptbuilder), [OpenAIChatGenerator](https://docs.haystack.deepset.ai/docs/openaichatgenerator), [ToolInvoker](https://docs.haystack.deepset.ai/docs/toolinvoker)
- **Prerequisites**: You must have an [OpenAI API Key](https://platform.openai.com/api-keys) and be familiar with [creating pipelines](https://docs.haystack.deepset.ai/docs/creating-pipelines)




#### 📚 Useful Sources:
* [OpenAIChatGenerator Docs](https://docs.haystack.deepset.ai/docs/openaichatgenerator)
* [OpenAIChatGenerator API Reference](https://docs.haystack.deepset.ai/reference/generator-api#openaichatgenerator)
* [🧑‍🍳 Cookbooks](https://github.com/deepset-ai/haystack-cookbook/blob/main/notebooks/)

[OpenAI's function calling](https://platform.openai.com/docs/guides/function-calling) connects large language models to external tools. By providing a `tools` list with functions and their specifications to the OpenAI API calls, you can easily build chat assistants that can answer questions by calling external APIs or extract structured information from text.

### 2. ⚙️ Setting up the Development Environment

Install Haystack and [sentence-transformers](https://pypi.org/project/sentence-transformers/) using pip:

In [25]:
%%bash

pip install haystack-ai
pip install "sentence-transformers>=3.0.0"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Enable Telemetry [Optional]

Knowing you're using this tutorial helps us decide where to invest our efforts to build a better product but you can always opt out by commenting the following line. See [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry) for more details.

In [ ]:
# from haystack.telemetry import tutorial_running

# tutorial_running(40)

### 3. 🔐 OpenAI Setup
Save your OpenAI API key as an environment variable:

In [26]:
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")

### 4. 🧱 Document Embedding (Tariff Docs)


### Index Documents with a Pipeline

Create a pipeline to store the small example dataset in the [InMemoryDocumentStore](https://docs.haystack.deepset.ai/docs/inmemorydocumentstore) with their embeddings. You will use [SentenceTransformersDocumentEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformersdocumentembedder) to generate embeddings for your Documents and write them to the document store with the [DocumentWriter](https://docs.haystack.deepset.ai/docs/documentwriter).

After adding these components to your pipeline, connect them and run the pipeline.

> If you'd like to learn about preprocessing files before you index them to your document store, follow the [Preprocessing Different File Types](https://haystack.deepset.ai/tutorials/30_file_type_preprocessing_index_pipeline) tutorial.

In [27]:
from haystack import Pipeline, Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

# Initialize document store
document_store = InMemoryDocumentStore()

import os

In [28]:
# 📜 Executive Order sample
eo_doc = Document(
    content=(
        "Section 2: Effective April 2025, a 20% tariff will be applied to electronic components "
        "imported from select non-domestic regions. Section 4: HS Codes 8542.31 and 8542.33 are "
        "explicitly mentioned in the scope of this order."
    ),
    meta={"source": "EO_2025_Tariffs.pdf", "type": "executive_order"}
)

# 🏢 Internal Supplier/Product Data
internal_doc = Document(
    content=(
        "Product: ARM Cortex-M4 SoC\n"
        "HS Code: 8542.31.0000\n"
        "Supplier: Global Embedded Systems Ltd.\n"
        "Region of Origin: Southeast Asia\n"
        "Quantity: 5,000 units\n"
        "Unit Cost: $18.25"
    ),
    meta={"source": "Q1_supplier_list.csv", "type": "internal_supplier_data"}
)

# 📚 Tariff Schedule Entry
tariff_schedule_doc = Document(
    content=(
        "HS Code 8542.31.0000 refers to: Electronic integrated circuits as processors and controllers, "
        "whether or not combined with memory. Current duty rate: 0%. Subject to change under recent trade updates."
    ),
    meta={"source": "HTS_2025.csv", "type": "tariff_schedule"}
)

# 📰 Trade News Sample
news_doc = Document(
    content=(
        "Trade analysts report that recent executive orders may increase tariffs on selected categories of "
        "microelectronics, with potential implications for global suppliers in the embedded systems industry."
    ),
    meta={"source": "industry_news_march2025.txt", "type": "trade_news"}
)

# 🧺 Final list of documents
documents = [eo_doc, internal_doc, tariff_schedule_doc, news_doc]


### 5. Build a RAG Pipeline

Build a basic retrieval augmented generative pipeline with [SentenceTransformersTextEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformerstextembedder), [InMemoryEmbeddingRetriever](https://docs.haystack.deepset.ai/docs/inmemoryembeddingretriever), [ChatPromptBuilder](https://docs.haystack.deepset.ai/docs/chatpromptbuilder) and [OpenAIChatGenerator](https://docs.haystack.deepset.ai/docs/openaichatgenerator).

> For a step-by-step guide to create a RAG pipeline with Haystack, follow the [Creating Your First QA Pipeline with Retrieval-Augmentation](https://haystack.deepset.ai/tutorials/27_first_rag_pipeline) tutorial.

In [29]:
# Build pipeline
pipeline = Pipeline()
pipeline.add_component("embedder", SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2"))
pipeline.add_component("writer", DocumentWriter(document_store=document_store))
pipeline.connect("embedder.documents", "writer.documents")

# Run the pipeline to store embedded documents
pipeline.run(data={"embedder": {"documents": documents}})

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.04it/s]


{'writer': {'documents_written': 4}}

#### 💬 Build the Chat Agent

In [30]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retriever = InMemoryEmbeddingRetriever(document_store)
text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

from haystack import Pipeline
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.utils import Secret
template = [
    ChatMessage.from_user(
    """
    Given the following information, answer the question.

    Context:
    {% for document in documents %}
        {{ document.content }}
    {% endfor %}

    Question: {{question}}
    Answer:
    """
    )
]
prompt_builder = ChatPromptBuilder(template=template)
chat_generator = OpenAIChatGenerator(model="gpt-4o-mini", api_key=Secret.from_env_var("OPENAI_API_KEY"))

basic_rag_pipeline = Pipeline()

basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("chat_generator", chat_generator)

basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder")
basic_rag_pipeline.connect("prompt_builder.prompt", "chat_generator.messages")


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: ChatPromptBuilder
  - chat_generator: OpenAIChatGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> chat_generator.messages (List[ChatMessage])

### 6. 🔍 Create a Tool from a Function

In addition to the `rag_pipeline_tool`, create a new tool called `get_tariff_info` to be used to get  tariff information from the manually created database.

First, create a function that simulates an API call to tariff information database. Instead of passing parameters as JSON (like in the previous tool), use [`create_tool_from_function`](https://docs.haystack.deepset.ai/docs/tool#create_tool_from_function). This function requires additional details using the `Annotated` type to describe tool parameters. However, based on this information, `create_tool_from_function` can automatically infer the parameters and generate a JSON schema, so you don't need to define `parameters` separately.

In [33]:
from typing import Annotated, Literal
from haystack.tools import create_tool_from_function

TARIFF_DATA = {
    "8542.31": {
        "product": "Electronic integrated circuits - processors and controllers",
        "tariff_rate": "25%",
        "effective_date": "2025-04-01",
        "included_in_eo": True
    },
    "8542.33": {
        "product": "Electronic integrated circuits - amplifiers",
        "tariff_rate": "15%",
        "effective_date": "2025-04-01",
        "included_in_eo": True
    },
    "9018.39": {
        "product": "Medical diagnostic equipment",
        "tariff_rate": "0%",
        "effective_date": "2025-04-01",
        "included_in_eo": False
    }
}
def get_tariff_info(
    hs_code: Annotated[str, "The HS Code of the product (e.g., 8542.31)"] = "8542.31",
):
    """Returns tariff information for a given HS code."""
    data = TARIFF_DATA.get(hs_code)
    if data:
        return data
    else:
        return {
            "product": "Unknown product",
            "tariff_rate": "N/A",
            "effective_date": "N/A",
            "included_in_eo": False
        }

# Register as a Haystack Tool
TariffIQ_tool = create_tool_from_function(get_tariff_info)

### 7. ⚙️ Chat Flow Logic

#### Running OpenAIChatGenerator with Tools

To use the tool calling feature, you need to pass the list of tools to `OpenAIChatGenerator` as `tools`. 

Instruct the model to use provided tools with a system message and then provide a query that requires a tool call as a user message:

In [34]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.generators.utils import print_streaming_chunk

user_messages = [
    ChatMessage.from_system(
        "Use the provided tool to answer questions about tariffs. Do not make assumptions — ask for clarification if the request is ambiguous."
    ),
    ChatMessage.from_user("What is the tariff rate for HS Code 8542.31?")
]


In [35]:
# 💬 Initialize chat generator
chat_generator = OpenAIChatGenerator(
    model="gpt-4o-mini", 
    streaming_callback=print_streaming_chunk
)

# 🔄 Run chat with tool access
response = chat_generator.run(
    messages=user_messages,
    tools=[TariffIQ_tool]  # 👈 Pass your custom tool here
)

Next, use [ToolInvoker](https://docs.haystack.deepset.ai/docs/toolinvoker) to process `ChatMessage` object containing tool calls, invoke the corresponding tools and return the results as a list of `ChatMessage`.  

In [36]:
from haystack.components.tools import ToolInvoker

# Register your tool (you've already created this)
tool_invoker = ToolInvoker(tools=[TariffIQ_tool])

# If the model requested a tool call, run the tool and capture results
if response["replies"][0].tool_calls:
    tool_result_messages = tool_invoker.run(messages=response["replies"])["tool_messages"]
    print(f"🔧 Tool result messages: {tool_result_messages}")


🔧 Tool result messages: [ChatMessage(_role=<ChatRole.TOOL: 'tool'>, _content=[ToolCallResult(result="{'product': 'Electronic integrated circuits - processors and controllers', 'tariff_rate': '25%', 'effective_date': '2025-04-01', 'included_in_eo': True}", origin=ToolCall(tool_name='get_tariff_info', arguments={'hs_code': '8542.31'}, id='call_huJmjdM3GgUm1mdAC4CJGtxW'), error=False)], _name=None, _meta={})]


As the last step, run the `OpenAIChatGenerator` again with the tool call results and get the final answer.

In [37]:
# Combine the system/user messages, AI replies, and tool output
messages = user_messages + response["replies"] + tool_result_messages

# Run the chat generator again with updated messages and the tool registered
final_replies = chat_generator.run(messages=messages, tools=[TariffIQ_tool])["replies"]

# Print the final answer
print(f"💬 Final answer: {final_replies[0].text}")


The tariff rate for HS Code 8542.31, which pertains to electronic integrated circuits (processors and controllers), is 25%. This rate will be effective starting from April 1, 2025.💬 Final answer: The tariff rate for HS Code 8542.31, which pertains to electronic integrated circuits (processors and controllers), is 25%. This rate will be effective starting from April 1, 2025.


### 8. 🖥️ Launch with Gradio

## Building the Chat Agent

As you notice above, OpenAI Chat Completions API does not call the tool; instead, the model generates JSON that you can use to call the tool in your code. That's why, to build an end-to-end chat agent, you need to check if the OpenAI response is a `tool_calls` for every message. If so, you need to call the corresponding tool with the provided arguments and send the tool response back to OpenAI. Otherwise, append both user and messages to the `messages` list to have a regular conversation with the model. Let's build an application that handles all cases.

To build a nice UI for your application, you can use [Gradio](https://www.gradio.app/) that comes with a chat interface. Install `gradio`, run the code cell below and use the input box to interact with the chat application that has access to two tools you've created above. 

> Note: OpenAI models can sometimes hallucinate answers or tools and might not work as expected.

In [39]:
%%bash

pip install -U gradio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [40]:
import gradio as gr
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.tools import ToolInvoker

# 🔧 Import your tariff tool
# from your_module import TariffIQ_tool
tool_invoker = ToolInvoker(tools=[TariffIQ_tool])

# 💬 Setup Chat Generator with the tool
chat_generator = OpenAIChatGenerator(model="gpt-4o-mini", tools=[TariffIQ_tool])
response = None

# System instruction
messages = [
    ChatMessage.from_system(
        "Use the tool provided to answer questions about tariffs. Don't make assumptions about inputs. Ask for clarification if needed."
    )
]

# 🧠 Core chatbot logic
def chatbot_with_tc(message, history):
    global messages
    messages.append(ChatMessage.from_user(message))
    response = chat_generator.run(messages=messages)

    while True:
        # 🔁 If the model makes a tool call
        if response and response["replies"][0].tool_calls:
            tool_result_messages = tool_invoker.run(messages=response["replies"])["tool_messages"]
            messages += response["replies"] + tool_result_messages
            response = chat_generator.run(messages=messages)
        else:
            # ✅ Regular response
            messages.append(response["replies"][0])
            break

    return response["replies"][0].text

# 💬 Gradio chat app
demo = gr.ChatInterface(
    fn=chatbot_with_tc,
    type="messages",
    examples=[
        "What tariffs affect HS Code 8542.31?",
        "What are the products in our supply list?",
        "What is the tariff rate of Electronic integrated circuits",
    ],
    title="💼 TariffIQ — Ask About Trade Policies & Product Impact",
    theme=gr.themes.Ocean(),
)

# 🚀 Launch
demo.launch()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


## 🚀 Now It's Your Turn: Build Something Game-Changing

🎉 Congrats — you've just built a fully working TariffIQ agent powered by Haystack + OpenAI tools!

You're now ready to go from **tutorial** to **real-world impact** — and this is your moment to shine.

---

### 💡 What You Can Build at the Hackathon:

- 🏷️ **Smart HS Code Lookup**  
  Let users upload product lists and automatically detect affected tariffs.

- 📰 **Executive Order Explainer**  
  Ask natural language questions over recent government docs or trade updates.

- 📦 **Supply Chain Risk Analyzer**  
  Build a chatbot that identifies high-risk suppliers or SKUs based on trade changes.

- 💼 **SMB Advisor Bot**  
  Help small businesses understand how policy changes affect their import/export pipelines.

---

### 🔧 Tools You Can Use (Beyond This Notebook):
- 🧠 [Custom Data Indexing](https://haystack.deepset.ai/tutorials) (PDFs, spreadsheets, APIs)
- 🌐 [Cookbooks](https://haystack.deepset.ai/cookbook)
- [Model-Based Evaluation of RAG Pipelines](https://haystack.deepset.ai/tutorials/27_first_rag_pipeline)


---

### 🌟 Get Inspired. Get Building. Get Creative.
This is your sandbox — bring your ideas to life using Haystack’s modular framework and OpenAI’s tools.
To stay up to date on the latest Haystack developments, you can [sign up for our newsletter](https://landing.deepset.ai/haystack-community-updates) or [join Haystack discord community](https://discord.gg/Dr63fr9NDS).

Happy Hacking 🎉 🎉 🎉 
